# Neural Network for Model Identification and Calibration

Two NN are used in a two step process:
1. Identification of the correct model
2. Calibration of the model

In [2]:
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
from keras.models import Sequential
import scipy
import numpy as np
import pickle
import gzip

### Utils

In [3]:
def myinverse(x,ub,lb):
    res = np.zeros (len(ub))
    for i in range (len(ub)):
        res[i] = x[i] * (ub[i] - lb[i]) * 0.5 + (ub[i] + lb[i]) * 0.5

    return res

NumLayers=3

def elu(x):
    #Careful function ovewrites x
    ind=(x<0)
    x[ind]=np.exp(x[ind])-1
    return x

def eluPrime(y):
    # we make a deep copy of input x
    x=np.copy(y)
    ind=(x<0)
    x[ind]=np.exp(x[ind])
    x[~ind]=1
    return x

def NeuralNetwork_param(x,NNParameters):
    input1=x
    for i in range(NumLayers):
        input1=input1@NNParameters[i][0]+NNParameters[i][1]
        #Elu activation
        input1=elu(input1)
    #The output layer is linear
    i+=1
    
    return input1@NNParameters[i][0]+NNParameters[i][1]

def NeuralNetworkGradient_param(x,NNParameters,num_params):
    input1=x
    #Identity Matrix represents Jacobian with respect to initial parameters
    grad=np.eye(num_params)
    #Propagate the gradient via chain rule
    for i in range(NumLayers):
        input1=input1@NNParameters[i][0]+NNParameters[i][1]
        grad=grad@NNParameters[i][0]
        #Elu activation
        grad*=eluPrime(input1)
        input1=elu(input1)
    grad= grad@NNParameters[i+1][0]
    #grad stores all intermediate Jacobians, however only the last one is used here as output
    return grad

## Main Function
Function that finds the best model (between the 5 selected) and calibrates model parameters

In [4]:
def choice_calibrate(vol_surface):
    
    model_name=["rBergomi","Bergomi1Factor","Heston","VGSSD","NIGSSD"]
    num_params=[4,4,5,4,4]
    
    ub = [[0.16,4.,-0.100011,0.499998],
          [1.59997e-01,3.99999e+00,9.91655e+00,-7.11209e-06],
          [0.03999168,-0.1000106,0.99997076,0.19999979,9.99974068],
          [1.,1.,-0.2,0.6],
          [2.,10.,-0.1,0.6]]
    lb = [[0.0100133,0.300028,-0.949934,0.0250066],
          [1.00178e-02,5.00077e-01,6.01685e-05,-9.99552e-01],
          [1.00512775e-04,-9.49762936e-01,1.00139137e-02,1.32859527e-02,1.01770174e+00],
          [0.3,0.2,-2.,0.3],
          [0.1,0.2,-11.,0.3]]
    
    model_cat = Sequential()
    model_cat.add(Conv2D(32, kernel_size=(3, 3), activation="relu",input_shape=(8,11,1)))
    model_cat.add(MaxPooling2D(pool_size=(2, 2)))
    model_cat.add(Flatten())
    model_cat.add(Dense(32,activation = 'relu'))
    model_cat.add(Dense(32,activation = 'relu'))
    
    model_cat.add(Dense(5,activation = 'softmax'))
    
    model_cat.load_weights('ModelWeights/CategoriesNNWeights.h5')
    
    idx_vec = model_cat.predict(vol_surface.reshape(1,8,11,1))[0]
    idx = np.argmax(idx_vec)
        
    print("Model selected: " + model_name[idx] + " with a precision of " + str(round(idx_vec[idx]*100,2))+"%")
    
    scale2 = pickle.load(open('Scales/'+model_name[idx]+'Scaler.pkl','rb'))
    
    f ="NNParams/"+model_name[idx]+"NNParams.npy"
    NNParameters = np.load(f,allow_pickle=True)
    
    def NeuralNetwork (x):
        return NeuralNetwork_param(x, NNParameters)
    def NeuralNetworkGradient (x):
        return NeuralNetworkGradient_param(x, NNParameters,num_params[idx])
    
    vol_transform = scale2.transform(vol_surface.reshape(1,88))
    
    def CostFuncLS(x):
        return (NeuralNetwork(x)-vol_transform.reshape(88))
    def JacobianLS(x):
        return NeuralNetworkGradient(x).T
    
    init = np.zeros(num_params[idx])
    I=scipy.optimize.least_squares(CostFuncLS,init,JacobianLS,gtol=1E-10)
    params = myinverse(I.x,ub[idx],lb[idx])
    
    return params

### Using the Function
Function is tested with some volatility surfaces (extracted from 1 Factor Bergomi and Heston, as an example) to see how it works

In [5]:
vola_1F = np.array([0.57082 , 0.514579, 0.464086, 0.419805, 0.380473, 0.344589,
       0.311389, 0.281852, 0.257049, 0.237843, 0.226023, 0.543843,
       0.492388, 0.447256, 0.406304, 0.368623, 0.333868, 0.301949,
       0.273193, 0.248624, 0.229844, 0.217796, 0.506208, 0.460093,
       0.419304, 0.382369, 0.348428, 0.317079, 0.288218, 0.262055,
       0.239336, 0.221267, 0.208746, 0.479372, 0.436928, 0.399483,
       0.36565 , 0.334606, 0.305915, 0.279422, 0.255275, 0.234029,
       0.216599, 0.203765, 0.457889, 0.418489, 0.383774, 0.352503,
       0.323857, 0.297358, 0.272835, 0.250406, 0.230493, 0.213793,
       0.200949, 0.439638, 0.403046, 0.370797, 0.341742, 0.315159,
       0.290621, 0.267938, 0.247143, 0.228515, 0.212569, 0.199844,
       0.425869, 0.391526, 0.361265, 0.334034, 0.309157, 0.286223,
       0.265022, 0.245537, 0.227958, 0.212669, 0.200107, 0.418342,
       0.385302, 0.35621 , 0.330043, 0.306144, 0.284108, 0.263734,
       0.244992, 0.228022, 0.213127, 0.200685])
vola_Heston = np.array([0.30589313, 0.25938753, 0.26559668, 0.27087143, 0.27544738,
       0.27947933, 0.28307693, 0.28632039, 0.28927003, 0.29197227,
       0.29446356, 0.33525328, 0.33916105, 0.34243849, 0.34525766,
       0.34772891, 0.34992724, 0.35190585, 0.35370386, 0.35535087,
       0.35686979, 0.35827871, 0.37284728, 0.37518921, 0.37716199,
       0.37886544, 0.38036376, 0.38170068, 0.38290732, 0.3840066 ,
       0.3850159 , 0.38594871, 0.3868157 , 0.38674206, 0.38840322,
       0.38980446, 0.39101584, 0.39208247, 0.39303511, 0.39389566,
       0.39468027, 0.39540117, 0.3960679 , 0.39668797, 0.3938646 ,
       0.39514272, 0.39622152, 0.39715464, 0.39797665, 0.39871112,
       0.39937484, 0.39998019, 0.40053658, 0.40105131, 0.40153014,
       0.39811111, 0.3991496 , 0.40002644, 0.40078509, 0.40145358,
       0.40205102, 0.40259102, 0.40308363, 0.40353648, 0.40395548,
       0.40434533, 0.40097146, 0.40184407, 0.402581  , 0.40321874,
       0.40378077, 0.40428314, 0.40473728, 0.40515162, 0.40553255,
       0.40588504, 0.40621305, 0.40239454, 0.40318334, 0.40384957,
       0.40442617, 0.40493437, 0.40538865, 0.40579935, 0.40617407,
       0.40651861, 0.40683744, 0.40713414])


In [6]:
choice_calibrate(vola_1F)

Model selected: Bergomi1Factor with a precision of 99.98%


array([ 0.12967477,  1.80251279,  0.75704231, -0.84992837])

In [7]:
choice_calibrate(vola_Heston)

Model selected: Heston with a precision of 99.88%


array([ 0.03099589, -0.50318905,  0.13115458,  0.17309677,  8.77650975])

Adding noise to see if it still works

In [8]:
choice_calibrate(vola_1F+1e-3)

Model selected: Bergomi1Factor with a precision of 99.98%


array([ 0.13051258,  1.80262877,  0.76017385, -0.84992523])